In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
from itertools import product
import scipy.stats as ss

In [2]:
pd.options.display.max_rows = 100
pd.options.display.max_columns = 100

## Table of Contents <a class="anchor" id="ToT"></a>
* [BonusOneTime](#BonusOneTime)
* [WageGross](#WageGross)
* [LanguageLevelID](#LanguageLevelID)
* [PositionID](#PositionID)
* [PositionLevel](#PositionLevel)
* [IsTrainee](#IsTrainee)
* [CustomerID](#CustomerID)
* [ProjectID](#ProjectID)
* [Utilization](#Utilization)
* [IsInternalProject](#IsInternalProject)
* [HourVacation](#HourVacation)
* [OnSite](#OnSite)
* [CompetenceGroupID](#CompetenceGroupID)
* [HiringDate](#HiringDate)
* [OriginalFeatures](#OriginalFeatures)

Generated features:
* WageQuant (numerical) - quantile of wages in which current wage lies. Wages are grouped by (PositionID, PositionLevel).
* TimeWageRate (numerical) - quantile of rate of wage growth. Rates are grouped by (PositionID, PositionLevel).  
Wage doesn't become bigger.
* LastLevelPeriod (numerical) - number of months person is on the last level on last position.  
Too long on one level.
* RestLevel (ordinal) - how many levels employee has to the top level in the current position. Levels are grouped by PositionID.  
Don't have space to growth.
* MeanBonus (numerical) - quantile of mean bonuses in which mean bonus for current employee lies. Bonuses are grouped by (PositionID, PositionLevel).  
1, 2, 3 - number of employee's position. 1 - last position, 2 - previous before it and so on.  
0 - wasn't on position with this number.
* MonthsOnProject (numerical) - number of months spent on the last project.  
Too long on project, become bored.
* Posrate (numerical) - number of a position rate during the employee's career.  
If many rate, unstable.
* FormerPosMonths (numerical) - number of months spent on previous position.  
If spent long time on previous position and changed, may be searching for rate in the life. Unstable, can leave.  
0 - wasn't previous position or wasn't long on previous position.
* MeanHourVacation (numerical) - mean number of vacation hours per user.  
Employees who don't ret may born out quicklier.
* PosLangLevel (ordinal) - level of English relative to position mode level.  
Different positions demand different knowledge of English. This parameter shows suitness of English level for current position. Can be used to compare employees from all positions.  
100 - wasn't on position with this number.
* ProjPosCount (numerical) - number of people with same position on project.  
If a person is alone on the project, he may feel bored.  
Nan if ProjectID = NaN.

In [3]:
hist = pd.read_pickle('./datasets/soft/clean/hist.pkl')
emp = pd.read_pickle('./datasets/soft/clean/emp.pkl')

In [4]:
def get_quants_quant(cur_value, quants):
    if quants.iloc[2] == 0:
        return 0

    if cur_value < quants.iloc[0]:
        cur_quant = 1
    elif quants.iloc[0] <= cur_value < quants.iloc[1]:
        cur_quant = 2
    elif quants.iloc[1] <= cur_value < quants.iloc[2]:
        cur_quant = 3
    else:
        cur_quant = 4
    return cur_quant

def get_quantile(cur_value, values):
    quants = values.quantile([0.25,0.5,0.75])
    return get_quants_quant(cur_value, quants)

In [5]:
df = hist.groupby('EmployeeID').apply(lambda x: x.iloc[0:x.shape[0]-2])
df = df.reset_index(drop=True)
df = df.sort_values(by=['EmployeeID', 'Date'])

#### BonusOneTime <a class="anchor" id="BonusOneTime"></a>
[<- Back](#ToT)

In [6]:
# fBonusOneTime
half_bonus_rate = df.groupby('EmployeeID').apply(lambda x: (x.iloc[-min(6, x.shape[0])]['BonusOneTime'] != 0).sum() / min(6, x.shape[0]))
year_bonus_rate = df.groupby('EmployeeID').apply(lambda x: (x.iloc[-min(12, x.shape[0])]['BonusOneTime'] != 0).sum() / min(12, x.shape[0]))
bonus_rate = df.groupby('EmployeeID').apply(lambda x: (x['BonusOneTime'] != 0).sum() / x.shape[0])
time_prev_bonus = df.groupby('EmployeeID').apply(lambda x: (x.shape[0] - (np.where(x['BonusOneTime'] != 0)[0][-1] + 1)) if (x['BonusOneTime'] != 0).sum() > 0 else 10000)

In [7]:
df['MeanPositionBonus'] = df.groupby(['PositionID', 'PositionLevel'])['BonusOneTime'].transform(lambda x: x[x!=0].mean())
df['BonusBiggerMean'] = np.where(df['BonusOneTime'] > df['MeanPositionBonus'], 1, 0)
prev_bonus_bigger_mean = df.groupby('EmployeeID').apply(lambda x: x[x['BonusOneTime'] > 0].iloc[-1]['BonusBiggerMean']
                                                  if x[x['BonusOneTime'] > 0].shape[0] > 0 else 0)

In [8]:
df['BonusDifference'] = np.where(df['BonusOneTime'] > 0, df['BonusOneTime'] - df['MeanPositionBonus'], None)
df['BonusDiffPosMean'] = df.groupby(['PositionID', 'PositionLevel'])['BonusDifference'].transform(lambda x: x.mean() if x.count() > 0 else 0)
bonus_diff = df.groupby('EmployeeID').apply(lambda x: x[x['BonusOneTime'] > 0].iloc[-1]['BonusDifference']
                                                  if x[x['BonusOneTime'] > 0].shape[0] > 0 else x.iloc[-1]['BonusDiffPosMean'])

In [9]:
def get_pos_bonus_quant(x):
    cur_bonus = x['BonusOneTime']
    cur_quant = get_quants_quant(cur_bonus, x[['BonusQ1', 'BonusQ2', 'BonusQ3']])
    return cur_quant

df['BonusQ1'] = df.groupby(['PositionID', 'PositionLevel'])['BonusOneTime'].transform(lambda x: x[x!=0].quantile(0.2))
df['BonusQ2'] = df.groupby(['PositionID', 'PositionLevel'])['BonusOneTime'].transform(lambda x: x[x!=0].quantile(0.5))
df['BonusQ3'] = df.groupby(['PositionID', 'PositionLevel'])['BonusOneTime'].transform(lambda x: x[x!=0].quantile(0.8))
df['PosBonusQuant'] = df.apply(get_pos_bonus_quant, axis=1)

pos_bonus_quant = df.groupby('EmployeeID').apply(lambda x: x[x['BonusOneTime'] > 0].iloc[-1]['PosBonusQuant']
                                                  if x[x['BonusOneTime'] > 0].shape[0] > 0 else 0)

In [10]:
bonus_comparison = np.where(bonus_rate > 0, half_bonus_rate / bonus_rate, 0)
bonus_comparison = pd.Series(data=bonus_comparison,
                            index=half_bonus_rate.index)

In [11]:
def get_bonus_quants(x):
    mean_bonuses = x.groupby('EmployeeID').mean()['BonusOneTime']
    quants = mean_bonuses.quantile([0.25, 0.5, 0.75])
    return quants

# group by Position and calculate quants
bonus_quants = df.groupby(['PositionID', 'PositionLevel']).apply(get_bonus_quants)

# print(bonus_quants.head())

def get_emp_quants(x):
    # group by PositionID + PositionLevel with sort=False and mean()
    mean_bonuses = x.groupby(['PositionID', 'PositionLevel'], sort=False).mean('BonusOneTime')
    # calculate quant of mean bonus
    quants = list()
    for index, obs in mean_bonuses.iterrows():
        quant = get_quants_quant(obs['BonusOneTime'], bonus_quants.loc[index])
        quants.append(quant)
    quants.reverse()
    return tuple([x.name] + quants[:min(3, len(quants))] + [0]*(3 - len(quants)))

# group by employee
mean_bonus = df.groupby('EmployeeID').apply(get_emp_quants)
mean_bonus = pd.DataFrame(mean_bonus.tolist(), columns=['EmployeeID', 'MeanBonus1', 'MeanBonus2', 'MeanBonus3'])
mean_bonus = mean_bonus.set_index('EmployeeID')
mean_bonus = mean_bonus.sort_index()

#### WageGross <a class="anchor" id="WageGross"></a>
[<- Back](#ToT)

In [12]:
# fWageGross
def get_half_wage_rate(x, period):
    last_wage = x.iloc[-1]['WageGross']
    earlier_month = min(period, x.shape[0])
    prev_wage = x.iloc[-earlier_month]['WageGross']
    rate = (last_wage - prev_wage) / earlier_month
    return rate

# too big percent
# tt = df.groupby('EmployeeID').apply(lambda x: x.shape[0] < 13)
# print(f'pct of < 12: {tt.sum() / tt.shape[0]}')

half_wage_rate = df.groupby('EmployeeID').apply(get_half_wage_rate, (6))
year_wage_rate = df.groupby('EmployeeID').apply(get_half_wage_rate, (12))
wage_rate = df.groupby('EmployeeID').apply(lambda x: (x.iloc[-1]['WageGross'] - x.iloc[0]['WageGross']) / x.shape[0])

In [13]:
df['PosGeoWageMean'] = df.groupby(['PositionID', 'PositionLevel', 'DevCenterID'])['WageGross'].transform('mean')
df['WageBiggerPGMean'] = np.where(df['WageGross'] > df['PosGeoWageMean'], 1, 0)
wage_bigger_pg_mean = df.groupby('EmployeeID').apply(lambda x: x.iloc[-1]['WageBiggerPGMean'])

In [14]:
df['PosStrWageMean'] = df.groupby(['PositionID', 'PositionLevel', 'SBUID'])['WageGross'].transform('mean')
df['WageBiggerPSMean'] = np.where(df['WageGross'] > df['PosStrWageMean'], 1, 0)
wage_bigger_ps_mean = df.groupby('EmployeeID').apply(lambda x: x.iloc[-1]['WageBiggerPSMean'])

In [15]:
wage_pg_diff = df.groupby('EmployeeID').apply(lambda x: x.iloc[-1]['WageGross'] - x.iloc[-1]['PosGeoWageMean'])
wage_ps_diff = df.groupby('EmployeeID').apply(lambda x: x.iloc[-1]['WageGross'] - x.iloc[-1]['PosStrWageMean'])

In [16]:
def get_pos_wage_quant(x):
    cur_wage = x['WageGross']
    cur_quant = get_quants_quant(cur_wage, x[['PosWageQ1', 'PosWageQ2', 'PosWageQ3']])
    return cur_quant

df['PosWageQ1'] = df.groupby(['PositionID', 'PositionLevel'])['WageGross'].transform(lambda x: x[x!=0].quantile(0.2))
df['PosWageQ2'] = df.groupby(['PositionID', 'PositionLevel'])['WageGross'].transform(lambda x: x[x!=0].quantile(0.5))
df['PosWageQ3'] = df.groupby(['PositionID', 'PositionLevel'])['WageGross'].transform(lambda x: x[x!=0].quantile(0.8))
df['PosWageQuant'] = df.apply(get_pos_wage_quant, axis=1)

pos_wage_quant = df.groupby('EmployeeID').apply(lambda x: x.iloc[-1]['PosWageQuant'])

In [17]:
def get_pos_lang_wage_quant(x):
    cur_wage = x['WageGross']
    cur_quant = get_quants_quant(cur_wage, x[['PosLangWageQ1', 'PosLangWageQ2', 'PosLangWageQ3']])
    return cur_quant

df['PosLangWageQ1'] = df.groupby(['PositionID', 'PositionLevel', 'LanguageLevelID'])['WageGross'].transform(lambda x: x[x!=0].quantile(0.2))
df['PosLangWageQ2'] = df.groupby(['PositionID', 'PositionLevel', 'LanguageLevelID'])['WageGross'].transform(lambda x: x[x!=0].quantile(0.5))
df['PosLangWageQ3'] = df.groupby(['PositionID', 'PositionLevel', 'LanguageLevelID'])['WageGross'].transform(lambda x: x[x!=0].quantile(0.8))
df['PosLangWageQuant'] = df.apply(get_pos_lang_wage_quant, axis=1)

pos_lang_wage_quant = df.groupby('EmployeeID').apply(lambda x: x.iloc[-1]['PosLangWageQuant'])
pos_lang_wage_quant.head()

EmployeeID
00116D71-E87D-4B64-A566-1F29B2A798A8    4
001502BB-B61A-404D-8A99-CC5DE5A4F985    1
001AAE0C-4EE9-43DB-BE7C-A6D08D061A9A    4
0034ECA4-0562-4AC7-A826-4AE81C64D69F    3
00384806-F711-41BA-A924-8F27E996F891    2
dtype: int64

In [18]:
wage_comparison = pd.Series(data=np.where(wage_rate > 0, half_wage_rate / wage_rate, 0), index=half_wage_rate.index)

In [19]:
# get last observes
last_df = df.groupby('EmployeeID').tail(1)
# for last observation
twage_growth = list()
for _, obs in last_df.iterrows():
    # filter by position (id, level)
    pos_df = df[(df['PositionID'] == obs['PositionID']) & (df['PositionLevel'] == obs['PositionLevel'])]
    # group by employee and find growth rates
    cur_rates = pos_df.groupby('EmployeeID').apply(lambda x: (max(x['WageGross']) 
                                                        - min(x['WageGross'])) / x.shape[0])
    cur_rate = cur_rates[obs['EmployeeID']]
    # find quantile
    cur_quant = get_quantile(cur_rate, cur_rates)
    twage_growth.append((obs['EmployeeID'], cur_quant))

# in quantiles
twage_growth = pd.DataFrame(twage_growth, columns=['EmployeeID', 'TimeWageRate'])
twage_growth = twage_growth.set_index('EmployeeID')
twage_growth = twage_growth.sort_index()

#### LanguageLevelID <a class="anchor" id="LanguageLevelID"></a>
[<- Back](#ToT)

In [20]:
# fLanguageLevelID
df['PosLangMode'] = df.groupby(['PositionID', 'PositionLevel'])['LanguageLevelID'].transform(lambda x: x.mode().mean())
df['LangBiggerPosMode'] = np.where(df['LanguageLevelID'] > df['PosLangMode'], 1, 0)

pos_lang_big_mode = df.groupby('EmployeeID').apply(lambda x: x.iloc[-1]['LangBiggerPosMode'])

In [21]:
def get_num_feat_rate(x, feat, period):
    last_val = x.iloc[-1][feat]
    earlier_month = min(period, x.shape[0])
    prev_val = x.iloc[-earlier_month][feat]
    rate = (last_val - prev_val) / earlier_month
    return rate

half_lang_rate = df.groupby('EmployeeID').apply(get_num_feat_rate, feat='LanguageLevelID', period=6)
year_lang_rate = df.groupby('EmployeeID').apply(get_num_feat_rate, feat='LanguageLevelID', period=12)
lang_rate = df.groupby('EmployeeID').apply(lambda x: (x.iloc[-1]['LanguageLevelID'] - x.iloc[0]['LanguageLevelID']) / x.shape[0])
lang_comparison = pd.Series(data=np.where(lang_rate > 0, half_lang_rate / lang_rate, 0), index=half_lang_rate.index)

In [22]:
df['PosProjLangMode'] = df.groupby(['PositionID', 'PositionLevel', 'ProjectID'])['LanguageLevelID'].transform(lambda x: x.mode().mean())
df['LangBiggerPPMode'] = np.where(df['LanguageLevelID'] > df['PosProjLangMode'], 1, 0)

pp_lang_big_mode = df.groupby('EmployeeID').apply(lambda x: x.iloc[-1]['LangBiggerPPMode'])

In [23]:
pos_lang_modes = df.groupby(['PositionID', 'PositionLevel']).aggregate(pd.Series.mode)['LanguageLevelID']
pos_lang_modes = pos_lang_modes.apply(lambda x: x if isinstance(x, np.int64) else np.mean(x))

def get_shift_lang(x):
    max_levels = 3
    pos_last = x.groupby(['PositionID', 'PositionLevel'], sort=False).tail(1)
    levels = list()
    for _, obs in pos_last.iterrows():
        mode = pos_lang_modes.loc[(obs['PositionID'], obs['PositionLevel'])]
        cur_level = obs['LanguageLevelID'] - mode
        levels.append(cur_level)
    levels.reverse()
    return [x.name] + levels[:min(3, len(levels))] + (max_levels - len(levels)) * [100]

shift_lang = df.groupby('EmployeeID').apply(get_shift_lang)
shift_lang = pd.DataFrame(shift_lang.to_list(), columns=['EmployeeID', 'LangPosToMode1', 'LangPosToMode2', 'LangPosToMode3'])
shift_lang = shift_lang.set_index('EmployeeID')
shift_lang = shift_lang.sort_index()

#### PositionID <a class="anchor" id="PositionID"></a>
[<- Back](#ToT)

In [24]:
# fPositionID
df['PosPeriodMean'] = df.groupby('EmployeeID')['PositionID'].transform(lambda x: x.shape[0] / x.nunique())
mean_pos_period = df.groupby('EmployeeID').apply(lambda x: x.iloc[-1]['PosPeriodMean'])
# equality because people who doesn't switch position are stable
df['PosBiggerMean'] = np.where(df['MonthOnPosition'] >= df['PosPeriodMean'], 1, 0)
pos_big_mean = df.groupby('EmployeeID').apply(lambda x: x.iloc[-1]['PosBiggerMean'])

In [25]:
top_5_pos = df.groupby('EmployeeID').tail(1).groupby('PositionID')['EmployeeID'].count().sort_values(ascending=False).index.values[0:5]
emp_top_pos = df.groupby('EmployeeID').apply(lambda x: x.iloc[-1]['PositionID'] in top_5_pos)
emp_top_pos = emp_top_pos.astype(np.int64)

In [26]:
df['PosGeoNum'] = df.groupby(['Date', 'PositionID', 'DevCenterID'])['EmployeeID'].transform(lambda x: x.nunique())
pos_geo_num = df.groupby('EmployeeID').apply(lambda x: x.iloc[-1]['PosGeoNum'])
df['PosStrNum'] = df.groupby(['Date', 'PositionID', 'SBUID'])['EmployeeID'].transform(lambda x: x.nunique())
pos_str_num = df.groupby('EmployeeID').apply(lambda x: x.iloc[-1]['PosStrNum'])

In [27]:
df['PosLevelGeoNum'] = df.groupby(['Date', 'PositionID', 'PositionLevel', 'DevCenterID'])['EmployeeID'].transform(lambda x: x.nunique())
pos_level_geo_num = df.groupby('EmployeeID').apply(lambda x: x.iloc[-1]['PosLevelGeoNum'])

In [28]:
def get_cat_feat_rate(x, feat, period):
    earlier_month = min(period, x.shape[0])
    num_rate = x.iloc[-earlier_month:][feat].nunique()
    rate = num_rate / earlier_month
    return rate

half_pos_rate = df.groupby('EmployeeID').apply(get_cat_feat_rate, feat='PositionID', period=6)
year_pos_rate = df.groupby('EmployeeID').apply(get_cat_feat_rate, feat='PositionID', period=12)
pos_rate = df.groupby('EmployeeID').apply(lambda x: x['PositionID'].nunique() / x.shape[0])
pos_rate_comparison = half_pos_rate / pos_rate

In [29]:
def get_former_pos_months(x):
    search_threshold = 3
    bore_threshold = 6
    positions = x['PositionID'].unique()
    if positions.size < 2:
        return 0
    former_pos_months = (x['PositionID'] == positions[-2]).sum()
    if former_pos_months < bore_threshold:
        return 0
    cur_pos_months = (x['PositionID'] == positions[-1]).sum()
    if cur_pos_months > search_threshold:
        return 0
    return former_pos_months

form_months = df.groupby('EmployeeID').apply(get_former_pos_months)
form_months = form_months.sort_index()
form_months = form_months.rename('FormerPosMonths')

In [30]:
# doesn't have a matter because constantly = 1
# df['PosLevelMean'] = df.groupby('PositionID')['PositionLevel'].transform(lambda x: x.nunique())
# df['PosLevBiggerMean'] = np.where(df['PositionLevel'] >= df['PosLevelMean'], 1, 0)
# pos_lev_big_mean = df.groupby('EmployeeID').apply(lambda x: x.iloc[-1]['PosLevBiggerMean'])

#### PositionLevel <a class="anchor" id="PositionLevel"></a>
[<- Back](#ToT)

In [31]:
# fPositionLevel
# all_lev_growth = df.groupby(['EmployeeID', 'PositionID']).apply(lambda x: (x.iloc[-1]['PositionLevel'] - x.iloc[0]['PositionLevel']) / x.shape[0])#.groupby('EmployeeID').mean()
# all_lev_growth#[all_lev_growth > 0]

# nobody improves his position
# all_lev_growth = df.groupby(['EmployeeID', 'PositionID']).apply(lambda x: x['PositionLevel'].nunique() > 1)#.groupby('EmployeeID').mean()
# all_lev_growth.sum()#[all_lev_growth > 0]

In [32]:
last_level_period = hist.groupby('EmployeeID').apply(lambda x: list(x['PositionLevel'] == x['PositionLevel'].iloc[-1]).count(True))
last_level_period.sort_index()
last_level_period = last_level_period.rename('LastLevelPeriod')

In [33]:
last_df = df.groupby('EmployeeID').tail(1)
rest_level = last_df.groupby('PositionID').apply(lambda x: pd.DataFrame({'EmployeeID': x['EmployeeID'], 
                                                                         'RestLevel': max(x['PositionLevel']) - x['PositionLevel']}))
rest_level.reset_index(drop=True, inplace=True)
rest_level = rest_level.set_index('EmployeeID')
rest_level = rest_level.sort_index()

#### IsTrainee <a class="anchor" id="IsTrainee"></a>
[<- Back](#ToT)

In [34]:
# fIsTrainee
was_trainee = df.groupby('EmployeeID').apply(lambda x: 1 in x['IsTrainee'].values)

#### CustomerID <a class="anchor" id="CustomerID"></a>
[<- Back](#ToT)

In [35]:
# CustomerID
df['CustTimeMean'] = df.groupby('CustomerID')['EmployeeID'].transform(lambda x: x.shape[0] / x.nunique())
cust_time_mean = df.groupby('EmployeeID').apply(lambda x: x.iloc[-1]['CustTimeMean'])
df['CustTime'] = df.groupby('CustomerID')['EmployeeID'].transform(lambda x: np.arange(1, x.size + 1))
df['CustTimeBiggerMean'] = np.where(df['CustTime'] > df['CustTimeMean'], 1, 0)
cust_time_big_mean = df.groupby('EmployeeID').apply(lambda x: x.iloc[-1]['CustTimeBiggerMean'])

In [36]:
emp['Dismissed'] = (~emp['DismissalDate'].isna()).astype('int64')
df = df.merge(emp[['EmployeeID', 'Dismissed']], on='EmployeeID')

In [37]:
# df = df.drop('Dismissed_x', axis=1)
# df = df.rename(columns={'Dismissed_y': 'Dismissed'})

In [38]:
cust_dism_emp = df.groupby('EmployeeID').apply(lambda x: x.iloc[-1]['CustomerID'] if x.iloc[-1]['Dismissed'] == 1 else None)
cust_dism_emp = cust_dism_emp.rename('CustomerID').reset_index()
cust_dism_emp = cust_dism_emp.groupby('CustomerID').count().rename(columns={'EmployeeID': 'DismCust'}).reset_index()
df = df.merge(cust_dism_emp, on='CustomerID', how='left')
df['DismCust'] = df['DismCust'].fillna(0)
cust_dism_num = df.groupby('EmployeeID').apply(lambda x: x.iloc[-1]['DismCust'])

In [39]:
df['CustNumProjs'] = df.groupby('CustomerID')['ProjectID'].transform(lambda x: x.nunique())
cust_num_projs = df.groupby('EmployeeID').apply(lambda x: x.iloc[-1]['CustNumProjs'])

In [40]:
df['CustNumEmps'] = df.groupby('CustomerID')['EmployeeID'].transform(lambda x: x.nunique())
cust_num_emps = df.groupby('EmployeeID').apply(lambda x: x.iloc[-1]['CustNumEmps'])
df['CustDismRate'] = df['DismCust'] / df['CustNumEmps']
cust_dism_rate = df.groupby('EmployeeID').apply(lambda x: x.iloc[-1]['CustDismRate'])

#### ProjectID <a class="anchor" id="ProjectID"></a>
[<- Back](#ToT)

In [41]:
# fProjectID
emp_proj_period_mean = df.groupby('EmployeeID')['ProjectID'].apply(lambda x: (~x.isna()).sum() / x.nunique() if x.nunique() > 0 else None).mean()
df['MonthOnProject'] = df.groupby(['EmployeeID', 'ProjectID'])['EmployeeID'].transform(lambda x: np.arange(1, x.size + 1))
proj_period_big_mean = df.groupby('EmployeeID').apply(lambda x: int(x.iloc[-1]['MonthOnProject'] > emp_proj_period_mean))

In [42]:
df['EmpOnProj'] = df.groupby(['Date', 'ProjectID'])['EmployeeID'].transform('nunique')
num_emp_proj = df.groupby('EmployeeID').apply(lambda x: x.iloc[-1]['EmpOnProj'])
num_emp_proj = num_emp_proj.fillna(1)

In [43]:
half_proj_rate = df.groupby('EmployeeID').apply(get_cat_feat_rate, feat='ProjectID', period=6)
year_proj_rate = df.groupby('EmployeeID').apply(get_cat_feat_rate, feat='ProjectID', period=12)
all_proj_rate = df.groupby('EmployeeID').apply(lambda x: x['ProjectID'].nunique() / x.shape[0])
proj_rate_comparison = pd.Series(data=np.where(all_proj_rate > 0, half_proj_rate / all_proj_rate, 0), index=half_proj_rate.index)

In [44]:
df['ProjPosWageMean'] = df.groupby(['ProjectID', 'PositionID', 'PositionLevel'])['WageGross'].transform('mean')
df['PosWageMean'] = df.groupby(['PositionID', 'PositionLevel'])['WageGross'].transform('mean')
df['ProjPosWageDiff'] = np.where(~df['ProjPosWageMean'].isna(), df['WageGross'] - df['ProjPosWageMean'], df['WageGross'] - df['PosWageMean'])
proj_pos_wage_diff = df.groupby('EmployeeID').apply(lambda x: x.iloc[-1]['ProjPosWageDiff'])

In [45]:
proj_period = df.groupby('EmployeeID').apply(lambda x: (x['ProjectID'] == x['ProjectID'].iloc[-1]).sum())
proj_period = proj_period.sort_index()
proj_period = proj_period.rename('MonthsOnProject')

In [46]:
pos_proj = df.groupby(['ProjectID', 'PositionID'])

def get_last_dates(x):
    dates = x.groupby('EmployeeID').tail(1)[['EmployeeID', 'Date']]
    return dates

emp_dates = pos_proj.apply(get_last_dates)

def get_date_count(x):
    cur_dates = emp_dates.loc[x.name]
    emp_counts = list()
    for idx, obs in cur_dates.iterrows():
        cur_count = (x['Date'] == obs['Date']).sum()
        emp_counts.append((obs['EmployeeID'], obs['Date'], cur_count))
    return pd.DataFrame(emp_counts, columns=['EmployeeID', 'Date', 'ProjPosCount'])

date_count = pos_proj.apply(get_date_count)
date_count = date_count.reset_index(drop=True)
date_count = date_count.sort_values(['EmployeeID', 'Date'])

def get_emp_counts(x):
    max_counts = 3
    counts = list(x['ProjPosCount'].iloc[::-1])
    return [x.name] + counts[:min(3, len(counts))] + [0] * (max_counts - len(counts))

emp_counts = date_count.groupby('EmployeeID').apply(get_emp_counts)
emp_counts = pd.DataFrame(emp_counts.to_list(), columns=['EmployeeID', 'ProjPosCount1', 'ProjPosCount2', 'ProjPosCount3'])
proj_pos_count = emp_counts.merge(emp[['EmployeeID']], how='right', on='EmployeeID')
proj_pos_count = proj_pos_count.set_index('EmployeeID').sort_index().fillna(1)

#### Utilization <a class="anchor" id="Utilization"></a>
[<- Back](#ToT)

In [47]:
# fUtilization
df['UtilizedBiggerMean'] = np.where(df['Utilization'] > df.groupby(['PositionID'])['Utilization'].transform('mean'), 1, 0)
util_big_mean = df.groupby('EmployeeID').apply(lambda x: 1 if (x.iloc[-3:]['UtilizedBiggerMean'] == 1).sum() == 3 else 0)

In [48]:
half_util_rate = df.groupby('EmployeeID').apply(lambda x: x.iloc[-min(6, x.shape[0]):]['Utilization'].mean())
year_util_rate = df.groupby('EmployeeID').apply(lambda x: x.iloc[-min(12, x.shape[0]):]['Utilization'].mean())
all_util_rate = df.groupby('EmployeeID').apply(lambda x: x['Utilization'].mean())
util_rate_comparison = pd.Series(data=np.where(all_util_rate > 0, half_util_rate / all_util_rate, 0), index=half_util_rate.index)

#### IsInternalProject <a class="anchor" id="IsInternalProject"></a>
[<- Back](#ToT)

In [49]:
int_proj_per = df.groupby('EmployeeID').apply(lambda x: x['IsInternalProject'].sum())
int_proj_now = df.groupby('EmployeeID').apply(lambda x: x.iloc[-1]['IsInternalProject'])
int_proj_last_pos = df.groupby('EmployeeID').apply(lambda x: np.int64(x[x['PositionID'] == x.iloc[-1]['PositionID']]['IsInternalProject'].sum() > 0))

#### HourVacation <a class="anchor" id="HourVacation"></a>
[<- Back](#ToT)

In [50]:
# fHourVacation
is_half_vacat = df.groupby('EmployeeID').apply(lambda x: np.int64(x.iloc[-min(6, x.shape[0]):]['HourVacation'].sum() > 0))
is_year_vacat = df.groupby('EmployeeID').apply(lambda x: np.int64(x.iloc[-min(12, x.shape[0]):]['HourVacation'].sum() > 0))
half_vacat_rate = df.groupby('EmployeeID').apply(lambda x: (x.iloc[-min(6, x.shape[0]):]['HourVacation'] > 0).sum() / 6)
year_vacat_rate = df.groupby('EmployeeID').apply(lambda x: (x.iloc[-min(12, x.shape[0]):]['HourVacation'] > 0).sum() / 12)
all_vacat_rate = df.groupby('EmployeeID').apply(lambda x: (x['HourVacation'] > 0).sum() / x.shape[0])
vacat_compar = pd.Series(data=np.where(all_vacat_rate > 0, half_vacat_rate / all_vacat_rate, 0), index=half_vacat_rate.index)

In [51]:
mean_vacation = df.groupby('EmployeeID')['HourVacation'].mean()
mean_vacation = mean_vacation.sort_index()
mean_vacation = mean_vacation.rename('MeanHourVacation')

#### OnSite <a class="anchor" id="OnSite"></a>
[<- Back](#ToT)

In [52]:
# fOnSite
is_quart_onsite = df.groupby('EmployeeID').apply(lambda x: np.int64(x.iloc[-min(3, x.shape[0]):]['OnSite'].sum() > 0))
half_onsite_rate = df.groupby('EmployeeID').apply(lambda x: x.iloc[-min(6, x.shape[0]):]['OnSite'].sum() / 6)
year_onsite_rate = df.groupby('EmployeeID').apply(lambda x: x.iloc[-min(12, x.shape[0]):]['OnSite'].sum() / 12)
all_onsite_rate = df.groupby('EmployeeID').apply(lambda x: x['OnSite'].sum() / x.shape[0])
onsite_compar = pd.Series(data=np.where(all_onsite_rate > 0, half_onsite_rate / all_onsite_rate, 0), index=half_onsite_rate.index)

In [53]:
df['OnSitePosMean'] = df.groupby(['PositionID'])['OnSite'].transform('mean')
onsite_pos_big_mean = df.groupby('EmployeeID').apply(lambda x: np.int64(x[x['PositionID'] == x.iloc[-1]['PositionID']]['OnSite'].mean() > x.iloc[-1]['OnSitePosMean']))

#### CompetenceGroupID <a class="anchor" id="CompetenceGroupID"></a>
[<- Back](#ToT)

In [54]:
# fCompetenceGroupID
df['CompGeoNum'] = df.groupby(['Date', 'DevCenterID', 'CompetenceGroupID'])['EmployeeID'].transform(lambda x: x.nunique())
comp_geo_num = df.groupby('EmployeeID').apply(lambda x: x.iloc[-1]['CompGeoNum'])

#### HiringDate <a class="anchor" id="HiringDate"></a>
[<- Back](#ToT)

In [55]:
def month_diff(a, b):
    return 12 * (a.dt.year - b.dt.year) + (a.dt.month - b.dt.month)

emp['HiringDate'] = pd.to_datetime(emp['HiringDate'], format='%m/%d/%y')
emp['DismissalDate'] = pd.to_datetime(emp['DismissalDate'], format='%m/%d/%y')
emp['WorkingPeriod'] = month_diff(emp['DismissalDate'].fillna(pd.Timestamp('20190201')), emp['HiringDate'])
work_period = emp.copy()
work_period = emp.set_index('EmployeeID')['WorkingPeriod']

#### OriginalFeatures <a class="anchor" id="OriginalFeatures"></a>
[<- Back](#ToT)

In [56]:
orig_features = df.groupby('EmployeeID').apply(lambda x: x.iloc[-1][['DevCenterID', 'SBUID', 'PositionID', 
                                                                     'PositionLevel', 'LanguageLevelID', 'Utilization', 
                                                                     'CompetenceGroupID', 'FunctionalOfficeID',
                                                                     'PaymentTypeId', 'APM', 'MonthOnSalary']])

In [57]:
gen_features = [half_bonus_rate.rename('HalfBonusRate'),
               year_bonus_rate.rename('YearBonusRate'),
               bonus_rate.rename('BonusRate'),
               time_prev_bonus.rename('TimeToPrevBonus'),
               prev_bonus_bigger_mean.rename('PrevBonusBiggerMean'),
               bonus_diff.rename('BonusPosMeanDiff'),
               pos_bonus_quant.rename('PosBonusQuant'),
               bonus_comparison.rename('BonusCompar').replace([np.inf, -np.inf], 1000),
               mean_bonus,
               
               half_wage_rate.rename('HalfWageRate'),
               year_wage_rate.rename('YearWageRate'),
               wage_rate.rename('WageRate'),
               wage_bigger_pg_mean.rename('WageBiggerPosGeoMean'),
               wage_bigger_ps_mean.rename('WageBiggerPosStrMean'),
               wage_pg_diff.rename('WagePosGeoDiff'),
               wage_ps_diff.rename('WagePosStrDiff'),
               pos_wage_quant.rename('WagePosQuant'),
               pos_lang_wage_quant.rename('WagePosLangQuant'),
               wage_comparison.rename('WageCompar').replace([np.inf, -np.inf], 1000),
               twage_growth,
               proj_pos_wage_diff.rename('WageProjPosDiff'),
               
               pos_lang_big_mode.rename('LangPosBiggerMode'),
               half_lang_rate.rename('HalfLangRate'),
               year_lang_rate.rename('YearLangRate'),
               lang_rate.rename('LangRate'),
               lang_comparison.rename('LangCompar').replace([np.inf, -np.inf], 1000),
               pp_lang_big_mode.rename('LangPosProjBiggerMode'),
               shift_lang,
               
               mean_pos_period.rename('PosPerMean'),
               pos_big_mean.rename('PosPerBiggerMean'),
               emp_top_pos.rename('IsTop5Pos'),
               pos_geo_num.rename('PosGeoNum'),
               pos_str_num.rename('PosStrNum'),
               pos_level_geo_num.rename('PosLevGeoNum'),
               half_pos_rate.rename('HalfPosRate'),
               year_pos_rate.rename('YearPosRate'),
               pos_rate.rename('PosRate'),
               pos_rate_comparison.rename('PosRateCompar').replace([np.inf, -np.inf], 1000),
               form_months.rename('FormPosMonths'),
               
               last_level_period,
               rest_level,
               
               was_trainee.rename('WasTrainee'),
               
               cust_time_mean.rename('CustTimeMean'),
               cust_time_big_mean.rename('CustTimeBiggerMean'),
               cust_dism_num.rename('CustDismNum'),
               cust_num_projs.rename('CustNumProj'),
               cust_num_emps.rename('CustNumEmp'),
               cust_dism_rate.rename('CustDismRate'),
               
               proj_period_big_mean.rename('ProjPerBiggerMean'),
               num_emp_proj.rename('ProjNumEmp'),
               half_proj_rate.rename('HalfProjRate'),
               year_proj_rate.rename('YearProjRate'),
               all_proj_rate.rename('ProjRate'),
               proj_rate_comparison.rename('ProjRateCompar').replace([np.inf, -np.inf], 1000),
               proj_period,
               proj_pos_count,
               
               util_big_mean.rename('3MonthsUtilBiggerMean'),
               half_util_rate.rename('HalfUtilRate'),
               year_util_rate.rename('YearUtilRate'),
               all_util_rate.rename('UtilRate'),
               util_rate_comparison.rename('UtilRateCompar').replace([np.inf, -np.inf], 1000),
               
               int_proj_per.rename('IntProjTime'),
               int_proj_now.rename('IsIntProjNow'),
               int_proj_last_pos.rename('IsIntProjLastPos'),
               
               is_half_vacat.rename('IsHalfVac'),
               is_year_vacat.rename('IsYearVac'),
               half_vacat_rate.rename('HalfVacRate'),
               year_vacat_rate.rename('YearVacRate'),
               all_vacat_rate.rename('VacRate'),
               vacat_compar.rename('VacRateCompar').replace([np.inf, -np.inf], 1000),
               mean_vacation,
               
               is_quart_onsite.rename('IsQuartOnSite'),
               half_onsite_rate.rename('HalfOnSiteRate'),
               year_onsite_rate.rename('YearOnSiteRate'),
               all_onsite_rate.rename('OnSiteRate'),
               onsite_compar.rename('OnSiteRateCompar').replace([np.inf, -np.inf], 1000),
               onsite_pos_big_mean.rename('OnSitePosBiggerMean'),
               comp_geo_num.rename('CompGeoNum'),
               
               work_period,
               
               orig_features]

In [58]:
feat = emp[['EmployeeID']].copy()
feat['Dismissed'] = (~emp['DismissalDate'].isna()).astype('int64')
feat = feat.set_index('EmployeeID')
feat = feat.sort_index()

In [59]:
import functools as ft

feat = ft.reduce(lambda left, right: left.join(right), [feat] + gen_features)

feat.head()

,Dismissed,HalfBonusRate,YearBonusRate,BonusRate,TimeToPrevBonus,PrevBonusBiggerMean,BonusPosMeanDiff,PosBonusQuant,BonusCompar,MeanBonus1,MeanBonus2,MeanBonus3,HalfWageRate,YearWageRate,WageRate,WageBiggerPosGeoMean,WageBiggerPosStrMean,WagePosGeoDiff,WagePosStrDiff,WagePosQuant,WagePosLangQuant,WageCompar,TimeWageRate,WageProjPosDiff,LangPosBiggerMode,HalfLangRate,YearLangRate,LangRate,LangCompar,LangPosProjBiggerMode,LangPosToMode1,LangPosToMode2,LangPosToMode3,PosPerMean,PosPerBiggerMean,IsTop5Pos,PosGeoNum,PosStrNum,PosLevGeoNum,HalfPosRate,YearPosRate,PosRate,PosRateCompar,FormPosMonths,LastLevelPeriod,RestLevel,WasTrainee,CustTimeMean,CustTimeBiggerMean,CustDismNum,CustNumProj,CustNumEmp,CustDismRate,ProjPerBiggerMean,ProjNumEmp,HalfProjRate,YearProjRate,ProjRate,ProjRateCompar,MonthsOnProject,ProjPosCount1,ProjPosCount2,ProjPosCount3,3MonthsUtilBiggerMean,HalfUtilRate,YearUtilRate,UtilRate,UtilRateCompar,IntProjTime,IsIntProjNow,IsIntProjLastPos,IsHalfVac,IsYearVac,HalfVacRate,YearVacRate,VacRate,VacRateCompar,MeanHourVacation,IsQuartOnSite,HalfOnSiteRate,YearOnSiteRate,OnSiteRate,OnSiteRateCompar,OnSitePosBiggerMean,CompGeoNum,WorkingPeriod,DevCenterID,SBUID,PositionID,PositionLevel,LanguageLevelID,Utilization,CompetenceGroupID,FunctionalOfficeID,PaymentTypeId,APM,MonthOnSalary
EmployeeID,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
00116D71-E87D-4B64-A566-1F29B2A798A8,0,0.0,0.083333,0.166667,2,0,-1.688333e+02,2,0.0,3,4,0,0.014575,0.008425,0.005617,1,0,0.146193,0.000000,3,4,2.595000,2,0.000000,0,0.0,0.0,0.000000,0.0,0,-8.0,0.0,100.0,9.0,0,0,2,1,2,0.333333,0.166667,0.111111,3.000000,17,1,0,False,9.027778,0,8.0,22,108,0.074074,0,35.0,0.333333,0.250000,0.166667,2.0,1,1.0,1.0,2.0,1,1.0,0.955950,0.908411,1.100823,0,0,0,1,1,0.666667,0.583333,0.611111,1.090909,18.444444,0,0.0,0.0,0.0,0.0,0,12,68,3,284,222,2,11,1.0,15,1,22,0,4
001502BB-B61A-404D-8A99-CC5DE5A4F985,1,0.0,0.000000,0.000000,10000,0,3.591133e+00,0,0.0,3,0,0,0.000000,0.000000,0.000000,0,0,-0.343076,-0.235943,1,1,0.000000,2,-0.360959,1,0.0,0.0,0.000000,0.0,0,9.0,100.0,100.0,8.0,1,1,114,5,114,0.166667,0.125000,0.125000,1.333333,0,10,0,False,9.600846,0,360.0,698,2836,0.126939,0,1.0,0.000000,0.000000,0.000000,0.0,0,1.0,1.0,1.0,0,0.0,0.000000,0.000000,0.000000,8,1,1,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.0,0.0,0.0,0.0,0,119,121,1,55,203,2,20,0.0,23,0,1,0,8
001AAE0C-4EE9-43DB-BE7C-A6D08D061A9A,0,0.0,0.000000,0.055556,4,1,2.416667e+02,4,0.0,3,0,0,0.000000,0.007456,0.004971,1,1,0.061230,0.058892,4,4,0.000000,4,0.000000,0,0.0,0.0,0.555556,0.0,0,0.0,100.0,100.0,18.0,1,0,1,1,1,0.166667,0.083333,0.055556,3.000000,0,20,0,False,9.600846,1,360.0,698,2836,0.126939,0,12.0,0.333333,0.166667,0.111111,3.0,2,1.0,1.0,0.0,0,0.0,0.000000,0.000000,0.000000,18,1,1,1,1,0.500000,0.500000,0.388889,1.285714,16.888889,0,0.0,0.0,0.0,0.0,0,780,47,1,120,89,2,15,0.0,0,-1,22,0,6
0034ECA4-0562-4AC7-A826-4AE81C64D69F,0,0.0,0.000000,0.166667,1,0,-1.763603e+02,2,0.0,4,4,3,0.020389,0.022958,0.023450,1,1,0.116545,0.172681,3,3,0.869461,4,0.155921,0,0.0,0.0,0.444444,0.0,0,0.0,8.0,0.0,6.0,1,0,7,2,7,0.166667,0.166667,0.166667,1.000000,0,1,0,False,8.131148,1,12.0,29,122,0.098361,0,19.0,0.333333,0.166667,0.111111,3.0,5,1.0,2.0,3.0,1,1.0,0.984467,0.953422,1.048853,0,0,0,1,1,0.500000,0.583333,0.500000,1.000000,12.888889,0,0.0,0.0,0.0,0.0,0,1,40,2,272,207,2,19,1.0,16,1,22,56,1
00384806-F711-41BA-A924-8F27E996F891,0,0.0,0.000000,0.000000,10000,0,3.777337e-14,0,0.0,3,0,0,0.000000,0.001137,0.000758,0,0,-0.039005,-0.017853,2,2,0.000000,2,-0.048954,1,0.0,0.0,0.000000,0.0,0,7.0,100.0,100.0,18.0,1,0,92,6,92,0.166667,0.083333,0.055556,3.000000,0,20,0,False,9.600846,1,360.0,698,2836,0.126939,0,1.0,0.000000,0.000000,0.000000,0.0,0,1.0,1.0,1.0,0,0.0,0.000000,0.000000,0.000000,18,1,1,0,0,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.0,0.0,0.0,0.0,0,449,45,1,55,206,1,18,0.0,17,1,22,0,7


In [60]:
feat.to_pickle('./datasets/soft/gen/feat_2m.pkl')
df.to_pickle('./datasets/soft/gen/df_2m.pkl')